This notebook merges each identified sequence-charge with the detected feature it came from, and saves the DF in percolator_df

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
ANALYSE_RECALIBRATED_FEATURES = True

In [5]:
MAXIMUM_Q_VALUE_PRE_RECAL = 0.01
MAXIMUM_Q_VALUE_POST_RECAL = 0.005

In [6]:
if ANALYSE_RECALIBRATED_FEATURES:
    PERCOLATOR_OUTPUT_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/recalibrated-percolator-output/190719_Hela_Ecoli.percolator.target.psms.txt'
    PERCOLATOR_STDOUT_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/recalibrated-percolator-output/recalibrated-percolator-stdout.log'
    FEATURE_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/recalibrated-features/{}-features.pkl'
    MAXIMUM_Q_VALUE = MAXIMUM_Q_VALUE_POST_RECAL
else:
    PERCOLATOR_OUTPUT_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/percolator-output/190719_Hela_Ecoli.percolator.target.psms.txt'
    PERCOLATOR_STDOUT_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/percolator-output/percolator-stdout.log'
    FEATURE_FILE = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/features/{}-features.pkl'
    MAXIMUM_Q_VALUE = MAXIMUM_Q_VALUE_PRE_RECAL

In [7]:
psms_df = pd.read_csv(PERCOLATOR_OUTPUT_FILE, sep='\t')

In [8]:
psms_df = psms_df[psms_df['percolator q-value'] <= MAXIMUM_Q_VALUE]

In [9]:
# count unique identifications
MAX_Q_VALUE_FOR_COUNTING_RESULTS = 0.01
df = pd.read_csv(PERCOLATOR_OUTPUT_FILE, sep='\t')
df = df[df['percolator q-value'] <= MAX_Q_VALUE_FOR_COUNTING_RESULTS]
print("there are {} proteins and {} peptides (max q-value is {})".format(len(df['protein id'].unique()), len(df.sequence.unique()), MAX_Q_VALUE_FOR_COUNTING_RESULTS))

there are 7904 proteins and 29057 peptides (max q-value is 0.01)


In [10]:
df['protein id'].unique()

array(['sp|P0AAI9|FABD_ECOLI', 'sp|P0AB71|ALF_ECOLI',
       'sp|Q13885|TBB2A_HUMAN,sp|Q9BVA1|TBB2B_HUMAN,sp|P68371|TBB4B_HUMAN,sp|P07437|TBB5_HUMAN,sp|Q13509|TBB3_HUMAN',
       ..., 'sp|Q8N4N8|KIF2B_HUMAN', 'sp|O15178|TBXT_HUMAN',
       'sp|O43439|MTG8R_HUMAN'], dtype=object)

In [11]:
psms_df[psms_df.file_idx == 0].sort_values(by='scan').head()

,file_idx,scan,charge,spectrum precursor m/z,spectrum neutral mass,peptide mass,percolator score,percolator q-value,percolator PEP,total matches/spectrum,sequence,protein id,flanking aa
85528,0,3,2,871.9096,1741.8046,1742.8120,0.863303,0.000003,2.177940e-08,3292.08,AMGIMNSFVNDIFER,"sp|Q5QNW6|H2B2F_HUMAN,sp|Q99879|H2B1M_HUMAN,sp...",KI
381962,0,4,2,901.9437,1801.8728,1802.8799,0.472076,0.000003,5.196051e-05,3292.08,GITINTSHVEYDTPTR,"sp|P0CE48|EFTU2_ECOLI,sp|P0CE47|EFTU1_ECOLI",RH
22826,0,5,3,801.0827,2400.2263,2401.2336,1.047681,0.000003,5.560437e-10,3292.08,ANDAAGDGTTTATVLAQAIITEGLK,sp|P0A6F5|CH60_ECOLI,KA
221,0,6,2,1093.0820,2184.1494,2128.1350,1.533166,0.000003,3.554143e-14,3292.08,IPISGIAGDQQAALFGQLCVK,sp|P0A6F3|GLPK_ECOLI,RE
282742,0,7,2,965.5842,1929.1539,1930.1615,0.595956,0.000003,4.442605e-06,3292.08,VTIAQGGVLPNIQAVLLPK,"sp|Q99878|H2A1J_HUMAN,sp|Q9BTM1|H2AJ_HUMAN,sp|...",KK


In [12]:
# use the recalibrated percolator output to find the mapping from percolator index to run
mapping = []
with open(PERCOLATOR_STDOUT_FILE) as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('INFO: Assigning index'):
            splits = line.split(' ')
            percolator_index = int(splits[3])
            comet_filename = splits[5]
            base_name = comet_filename.split('/')[-1].split('.')[0]
            mapping.append((percolator_index, base_name))

In [13]:
mapping

[(0, '190719_Hela_Ecoli_1to3_06-recalibrated'),
 (1, '190719_Hela_Ecoli_1to1_07-recalibrated'),
 (2, '190719_Hela_Ecoli_1to1_09-recalibrated'),
 (3, '190719_Hela_Ecoli_1to3_05-recalibrated'),
 (4, '190719_Hela_Ecoli_1to3_09-recalibrated'),
 (5, '190719_Hela_Ecoli_1to3_08-recalibrated'),
 (6, '190719_Hela_Ecoli_1to3_07-recalibrated'),
 (7, '190719_Hela_Ecoli_1to3_01-recalibrated'),
 (8, '190719_Hela_Ecoli_1to3_10-recalibrated'),
 (9, '190719_Hela_Ecoli_1to3_04-recalibrated'),
 (10, '190719_Hela_Ecoli_1to1_02-recalibrated'),
 (11, '190719_Hela_Ecoli_1to1_10-recalibrated'),
 (12, '190719_Hela_Ecoli_1to1_04-recalibrated'),
 (13, '190719_Hela_Ecoli_1to1_06-recalibrated'),
 (14, '190719_Hela_Ecoli_1to1_03-recalibrated'),
 (15, '190719_Hela_Ecoli_1to1_08-recalibrated'),
 (16, '190719_Hela_Ecoli_1to3_03-recalibrated'),
 (17, '190719_Hela_Ecoli_1to1_01-recalibrated'),
 (18, '190719_Hela_Ecoli_1to1_05-recalibrated'),
 (19, '190719_Hela_Ecoli_1to3_02-recalibrated')]

In [19]:
[m[0] for m in mapping if m[1] == '190719_Hela_Ecoli_1to1_04-recalibrated'][0]

12

In [11]:
# combine the features detected across the whole experiment into a single dataframe
df_l = []
for m in mapping:
    idx = m[0]
    filename = m[1]
    df = pd.read_pickle(FEATURE_FILE.format(filename))
    df['percolator_idx'] = idx
    df['batch'] = filename.split('_')[3]
    df_l.append(df)
# make a single df from the list of dfs
features_df = pd.concat(df_l, axis=0, sort=False)

In [12]:
features_df.columns

Index(['feature_id', 'charge', 'rt_apex', 'rt_peak_width', 'scan_apex',
       'scan_peak_width', 'intensity', 'precursor_id', 'monoisotopic_mass',
       'predicted_mass_error', 'recalibrated_monoisotopic_mass',
       'recalibrated_monoisotopic_mz', 'percolator_idx', 'batch'],
      dtype='object')

In [13]:
len(features_df[features_df.percolator_idx == 0])

149261

In [14]:
# merge the features with the experiment-wide percolator identifications
percolator_df = pd.merge(psms_df, features_df, how='left', left_on=['file_idx','scan'], right_on=['percolator_idx','feature_id'])
percolator_df['human'] = percolator_df['protein id'].str.contains('HUMAN')

In [15]:
len(percolator_df)

575632

percolator_df contains all the sequence-charge identifications across the experiment and the detected feature for each.

In [16]:
percolator_df.to_pickle('/Users/darylwilding-mcbride/Downloads/percolator_df.pkl')

In [17]:
pd.DataFrame(mapping, columns=['file_idx','run_name']).to_pickle('/Users/darylwilding-mcbride/Downloads/percolator_mapping_df.pkl')